In [1]:
import pandas as pd
data=pd.read_csv("C://Users//POPO//Desktop//train.csv")
test=pd.read_csv("C://Users//POPO//Desktop//test.csv")

In [2]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
datax=data.drop(["Ticket","Name","PassengerId","Survived"],axis=1)
testx=test.drop(["Ticket","Name","PassengerId"],axis=1)
datax=pd.concat([testx,datax])

In [4]:
datax.isnull().sum()

Pclass         0
Sex            0
Age          263
SibSp          0
Parch          0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [5]:
datax["Age"]=datax["Age"].fillna(datax["Age"].mean())
datax["Fare"]=datax["Fare"].fillna(datax["Fare"].mean())

In [6]:
datax = datax.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [7]:
datax.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [8]:
datax=pd.concat([datax.drop(["Pclass","Sex","Cabin","Embarked"],axis=1),pd.get_dummies(datax[["Pclass","Sex","Cabin","Embarked"]])],axis=1)

In [9]:
datax.head()

,Age,SibSp,Parch,Fare,Pclass,Sex_female,Sex_male,Cabin_A10,Cabin_A11,Cabin_A14,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,34.5,0,0,7.8292,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,47.0,1,0,7.0000,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,62.0,0,0,9.6875,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,27.0,0,0,8.6625,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22.0,1,1,12.2875,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
datay=data["Survived"]

In [11]:
print(testx.shape)
print(datax.shape)

(418, 8)
(1309, 196)


In [12]:
trainx=datax.iloc[418:,:]
testx=datax.iloc[0:418,:]

In [13]:
print(trainx.shape)
print(datay.shape)
print(testx.shape)

(891, 196)
(891,)
(418, 196)


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    trainx, datay, test_size=0.2, random_state=1)
from sklearn.neighbors import KNeighborsClassifier

# p=2 and metric='minkowski' means the Euclidean Distance
knn = KNeighborsClassifier(n_neighbors=10, p=2, metric='minkowski')

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('[KNN]')
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

[KNN]
Misclassified samples: 54
Accuracy: 0.70


In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_C = [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
param_gamma = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]

svm = SVC(random_state=0)

# set the param_grid parameter of GridSearchCV to a list of dictionaries
param_grid = [{'C': param_C, 
               'gamma': param_gamma, 
               'kernel': ['rbf']}]
gs = GridSearchCV(estimator=svm, 
                  param_grid=param_grid, 
                  scoring='accuracy')

gs = gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)

C:\Users\POPO\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8132022471910112
{'C': 10000.0, 'gamma': 0.0001, 'kernel': 'rbf'}


In [40]:
svm = SVC(kernel='rbf', random_state=0, gamma=0.0001, C=10000.0)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Misclassified samples: 37
Accuracy: 0.79


In [56]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=50000, 
                                random_state=1,
                                n_jobs=5)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)
print('Accuracy (forest): %.2f' % accuracy_score(y_test, y_pred))

Accuracy (forest): 0.77


In [50]:
from sklearn.tree import DecisionTreeClassifier

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=3, 
                              random_state=0)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Accuracy (tree): %.2f' % ((y_test == y_pred).sum() / y_test.shape[0]))

Misclassified samples: 35
Accuracy (tree): 0.80


In [65]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

importances = tree.feature_importances_
# get sort indices in descending order
indices = np.argsort(importances)[::-1]

for f in range(10):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            trainx.columns.values[indices[f]], 
                            importances[indices[f]]))


 1) Sex_female                     0.527845
 2) Pclass                         0.190712
 3) Age                            0.085921
 4) Cabin_C23 C25 C27              0.075302
 5) SibSp                          0.062569
 6) Fare                           0.057652
 7) Cabin_B79                      0.000000
 8) Cabin_C105                     0.000000
 9) Cabin_C104                     0.000000
10) Cabin_C103                     0.000000


In [33]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0,C=500)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Accuracy (tree): %.2f' % ((y_test == y_pred).sum() / y_test.shape[0]))

Misclassified samples: 38
Accuracy (tree): 0.79


C:\Users\POPO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [66]:
Survived=tree.predict(testx)

In [67]:
final=pd.concat([pd.DataFrame(test["PassengerId"]),pd.DataFrame(Survived)],axis=1)
final.columns=["PassengerId","Survived"]
final=final.set_index("PassengerId")

In [68]:
final.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [69]:
final.to_csv("C://Users//POPO//Desktop//Result.csv")